In [38]:
import numpy as np
from random import randint
import pandas as pd
from pickle import dump
import string
import re
from keras.preprocessing.text import Tokenizer 
from keras.utils.vis_utils import plot_model 
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences

In [2]:
def load_file(filename):
    file = open(filename,'r')
    data = file.read()
    file.close()
    return data

In [3]:
filename = 'republican_clean.txt'
data = load_file(filename)

In [4]:
def clean_text(file):
    file = file.replace('--',' ')
    
    tokens = data.split()
    
    re_punct = re.compile('[%s]'%re.escape(string.punctuation))
    cleaned_text = [re_punct.sub('',w) for w in tokens]
    cleaned_text = [word for word in cleaned_text if word.isalpha()]
    cleaned_text = [word.lower() for word in cleaned_text]
    
    return cleaned_text

In [5]:
cleaned_doc = clean_text(data)

In [6]:
print("Vocabulary Size : ", len(cleaned_doc))
print("Unique Vocab : ", len(set(cleaned_doc)))

Vocabulary Size :  118214
Unique Vocab :  7837


In [7]:
def create_sequences(length, cleaned_doc):
    # we'll keep each sequence of length 50, so we will input 50 words into the model at a time
    # and output one word
    
    # length here is set to 50, so start is at 51, and end is as at len(cleaned_doc)
    # so end becomes 118214
    
    # line will ultimately range from 0 : 50 which is 51 words in total, first 50 of which
    # would be input words and the last one (50th word) would be the output word
    start = length + 1
    end = len(cleaned_doc)
    sequences = []
    for i in range(start, end):
        line = cleaned_doc[i - start : i]
        
        line = ' '.join(line)
        
        sequences.append(line)
    return sequences

In [8]:
length = 50
sequences = create_sequences(length, cleaned_doc)

In [9]:
def sequence_to_file(sequences, filename):
    lines = '\n'.join(sequences)
    file = open(filename,'w')
    file.write(lines)
    file.close()

In [10]:
filename = 'republic_sequences.txt'
sequence_to_file(sequences, filename)

In [11]:
def generate_data(data):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(data)
    vocab_size = len(tokenizer.word_index)+1
    encoded = np.array(tokenizer.texts_to_sequences(data))
    X = np.array(encoded[:,:-1])
    y = np.array(encoded[:,-1])
    y = np.array(to_categorical(y, num_classes = vocab_size))
    return X,y,tokenizer, vocab_size

In [12]:
lines = load_file('republic_sequences.txt')
lines = lines.split('\n')

In [13]:
X,y,tokenizer, vocab_size = generate_data(lines)

In [14]:
sequence_length = X.shape[1]

In [15]:
print("X shape : ",X.shape)
print("y shape : ",y.shape)
print("Vocabulary Size : ",vocab_size)
print("Sequence length : ", X.shape[1])

X shape :  (118163, 50)
y shape :  (118163, 7838)
Vocabulary Size :  7838
Sequence length :  50


In [16]:
def make_model(vocab_size,sequence_length):
    model = Sequential()
    model.add(Embedding(vocab_size, output_dim = 50, input_length = sequence_length))
    model.add(LSTM(100, activation = 'relu', return_sequences= True))
    model.add(LSTM(100, activation = 'relu'))
    model.add(Dense(100, activation = 'relu'))
    model.add(Dense(vocab_size, activation = 'softmax'))
    model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer = 'adam')
    return model

In [17]:
model = make_model(vocab_size, sequence_length)

In [18]:
model.fit(X,y, batch_size = 128, epochs = 100)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/100





118163/118163 [==============================] - 148s 1ms/step - loss: 6.3859 - acc: 0.0568
Epoch 2/100
118163/118163 [==============================] - 145s 1ms/step - loss: 6.0553 - acc: 0.0699
Epoch 3/100
118163/118163 [==============================] - 144s 1ms/step - loss: 5.7584 - acc: 0.1038
Epoch 4/100
118163/118163 [==============================] - 146s 1ms/step - loss: 5.4621 - acc: 0.1323
Epoch 5/100
118163/118163 [==============================] - 146s 1ms/step - loss: 5.2583 - acc: 0.1464
Epoch 6/100
118163/118163 [==============================] - 146s 1ms/step - loss: 5.0892 - acc: 0.1592
Epoch 7/100
118163/118163 [==============================] - 146s 1ms/step - loss: 4.9391 - acc: 0.1700
Epoch 8/100
118163/118163 [==============================] - 146s 1ms/step - loss: 4.8008 - acc: 0.1783
Epoch 9/100
118163/118163 [==============================] - 146s 

In [19]:
model.save('language_model.h5')

In [23]:
dump(tokenizer,open('tokenizer.pkl','wb'))

In [29]:
lines = load_file('republic_sequences.txt')
lines = lines.split('\n')

In [35]:
sequence = lines[0]
sequence

'book i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was'

In [39]:
sequence_length

50

In [50]:
def generate_sequences(n_words, tokenizer, model, sequence_length, seed_text):
    input_text = seed_text
    results = []
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([input_text])[0]
        padded_text = pad_sequences([encoded], maxlen = sequence_length, padding = 'pre')
        y_hat = model.predict_classes(padded_text, verbose = 0)
        output_text = ''
        for word, index in tokenizer.word_index.items():
            if index == y_hat:
                output_text = word
                break
        input_text = input_text + ' ' + output_text
        results.append(output_text)
    return ' '.join(results)

In [57]:
seed_text = lines[randint(0, len(lines))]
n_words = 50
result = generate_sequences(n_words, tokenizer, model, sequence_length, seed_text)
print(seed_text)
print(result)

i replied now i meant that you should admit the utility and in this way as i thought i should escape from one of them and then there would remain only the possibility but that little attempt is detected and therefore you will please to give a defence of both well
but do you say that there was something who cannot succeed according to me socrates and we may reject as want to exceed the voice of the shepherd that justice gives us who are rich and seen in a few now that he said is the other hand the friction
